In [18]:
import os

In [19]:
%pwd

'd:\\MLproject\\First_project\\Project\\machine-learning-mlflow'

In [20]:
os.chdir("../")

In [21]:
%pwd

'd:\\MLproject\\First_project\\Project'

In [22]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path : Path
    test_data_path : Path
    model_name : str
    alpha : float
    l1_ratio : float
    target_column : str

In [23]:
from src.mlploject.constants import *
from src.mlploject.utils.common import read_yaml , create_directories

In [24]:
class ConfigarationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schem = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config (self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.Elasticnet
        schema = self.schem.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path= config.test_data_path,
            model_name= config.model_name,
            alpha= params.alpha,
            l1_ratio= params.l1_ratio,
            target_column= schema.name
        )
        
        return model_trainer_config

In [25]:
import pandas as pd
from sklearn.linear_model import ElasticNet
import joblib

In [26]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data([self.config.target_column])
        test_y = train_data([self.config.target_column])
        
        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        
        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [27]:
try:
    config = ConfigarationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer  = ModelTrainer(config= model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'

In [30]:
a = pd.read_csv("D:/MLproject/First_project/Project/machine-learning-mlflow/artifacts/data_transformation/train.csv")

In [31]:
a.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,9.4,0.430,0.24,2.8,0.092,14.0,45.0,0.99800,3.19,0.73,10.0,6
1,10.7,0.520,0.38,2.6,0.066,29.0,56.0,0.99577,3.15,0.79,12.1,7
2,7.5,0.755,0.00,1.9,0.084,6.0,12.0,0.99672,3.34,0.49,9.7,4
3,10.7,0.350,0.53,2.6,0.070,5.0,16.0,0.99720,3.15,0.65,11.0,8
4,8.2,0.380,0.32,2.5,0.080,24.0,71.0,0.99624,3.27,0.85,11.0,6


In [32]:
train = a.drop(['quality'],axis=1)

In [34]:
test = a['quality']

In [35]:
test

0       6
1       7
2       4
3       8
4       6
       ..
1194    5
1195    5
1196    6
1197    6
1198    6
Name: quality, Length: 1199, dtype: int64